# Trash End to End - SQL

In [ ]:
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobClient
import json 
import os
import pyodbc
import subprocess

In [ ]:
''' blobContainer create a name_list of blobs within container'''
''' params are container name (no full url) & storage conn string'''
def blobInContainer(connection_s,container_n):
    campaign_container = ContainerClient.from_connection_string(conn_str=connection_s, container_name=container_n)
    blob_list = campaign_container.list_blobs()
    blob_names_list = []
    for blob in blob_list:
        blob_names_list.append(blob.name)
    return blob_names_list

''' blobInfos provides basic information about a blob object'''
''' params are blob_name only (no full url) & storage conn string'''
def blobInfos(connection_s,container_n,blob_n):
    blob_video = BlobClient.from_connection_string(conn_str=connection_s,container_name=container_n, blob_name=blob_n)
    blob_video_url = blob_video.url
    blob_video_prop = blob_video.get_blob_properties()
    blob_video_prop_keys = blob_video_prop.keys()
    print("blob name:",blob_n)
    print("blob URL:",blob_video_url)
    print("blob properties:", blob_video_prop)
    print("blob properties keys:", blob_video_prop_keys)


''' downloadBlob from Azure to local file system'''
''' parameter is a blob client object from azure storage sdk'''
def downloadBlob(blobclient):
    with open("/tmp/"+blobclient.blob_name, "wb") as my_blob_dl:
        blob_data = blobclient.download_blob()
        blob_data.readinto(my_blob_dl)
    print("Blob %s downloaded" %blobclient.blob_name)
    print("Blob path: /tmp/%s" %blobclient.blob_name)


''' getPrediction function sends curl request to Surfrider AI'''
''' video name is the name of a locally downloaded video from Azure'''
''' video name is passed as an argument to curl_request script '''
''' curl_request script sends actual POST request to Surfrider AI'''
def getPrediction(video_name):
    curl_request_script = ['./curl_request_param.sh',video_name]
    output = []
    request_answer = subprocess.Popen(curl_request_script, stdout=subprocess.PIPE)
    i = 0
    for line in request_answer.stdout:
        print(line)
        output.append(line)
    return output


''' jsonPrediction cast a prediction from getPrediction function '''
''' from a list prediction to a string then dictionnary with json_loads '''
def jsonPrediction(pred):
    string_prediction = str(pred[0])[2:-3] #removing 2 x first and 3 last characters of pred
    json_prediction = json.loads(string_prediction)
    return json_prediction


''' getTrashLabel return label from a frame_to_box'''
def getTrashLabel(frame_2_box):
    return frame_2_box['label']

''' mapLabelTrashId is a switch that converts label to TrashId'''
''' param is label that comes from AI predictions dictionnary jsonPrediction'''
def mapLabel2TrashId(label):
    switcher = { 
    "Fishing or Hunting":"89B44BAA-69AA-4109-891A-128E012E7E07",
    "Food Packaging":"185FEFA2-EEF2-47A8-873E-26032A4BB3C3",
    "Unknown":"BB4DEA69-218A-40CC-A000-2AE17C37152C",
    "Industrial or Construction Debris":"2A863E38-E5D0-455F-87CE-2B75DA29F59A",
    "fragments":"ED401B92-DC24-44C0-A52A-34CE831092BF",
    "Agricultural Waste":"36B2AFEB-7A7C-44B5-A790-5E5C73BA144D",
    "others":"4BEC18FC-BC48-45B7-AFDA-6BA96BD80921",
    "Common Household Items":"C68E90CF-6E65-4474-BC60-72E1C8513F55",
    "plastic":"6961D0DB-928C-419E-9985-98EEEAF552C7",
    "bottles":"9780940B-D06C-4AAB-8003-AB914981E87A",
    "Drinking Bottles":"BCF549A8-AECD-4BC9-B9B8-B94A8F3758D5",
    "Unknown10":"BC7BB564-BE04-4B4B-9913-FF69780B93A6"
    } 
    return switcher.get(label, "nothing")


''' trashInsert execute INSERT request to dbo.Trash table'''
''' parameter to indsert is a trashTypeId defined in Trash_Type table'''
def trashInsert(cursor,cnxn,trashTypeId):
    cursor.execute("INSERT dbo.Trash (Id,CampaignId,Latitude,Longitude,TrashTypeId,Precision,AI_Version) VALUES (NEWID(),'8D21A132-CF4B-404E-B287-C40A2F12D305','50.797731', '2.179029', ? ,'0.95','0')",trashTypeId)
    cnxn.commit()

In [ ]:
def main():
    
    # blob storage connection string
    connection_string = os.getenv("CONN_STRING")
 
    # get list of blobs in container campaign0
    campaign_container_name = 'campaign0'
    #campaign_container_name = 'campaign1'
    blobs_campaign0 = blobInContainer(connection_string,campaign_container_name)
    print(blobs_campaign0)

    # get infos of blob video vid1-4K.MP4,vid1-4K2.MP4,vid1-4K3.mp4,vid1-4K4.mp4,19022020_Adour_6.mp4
#    blob_video_name = '19022020_Adour_6.mp4'
    blob_video_name = 'vid1-480p.mov'   
    blobInfos(connection_string,campaign_container_name,blob_video_name)

    # download blob video
    blob_video0 = BlobClient.from_connection_string(conn_str=connection_string,container_name=campaign_container_name, blob_name=blob_video_name)
    downloadBlob(blob_video0)

    # get predictions from AI on vid1-4K4.MP4
    prediction = getPrediction(blob_video_name)

    # cast prediction to JSON/Dictionnary format
    json_prediction = jsonPrediction(prediction)
    json_prediction

    # SQL connection
    server = os.getenv("SERVER")
    database = os.getenv("DATABASE")
    username = os.getenv("USERNAME")
    password = os.getenv("SQLPWD")
    driver= '{ODBC Driver 17 for SQL Server}'
    cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()

    # Executing INSERT for all trash detected in json_prediction from vid1-4K4.MP4
    for frame_to_box in json_prediction['detected_trash']:
        print(frame_to_box)
        trashLabel = getTrashLabel(frame_to_box)
        print(trashLabel)
        mapLabel = mapLabel2TrashId(trashLabel)
        print(mapLabel)
        trashInsert(cursor,cnxn,mapLabel)
    
    # Visualize main result within dbo.Trash Table
    cursor.execute("SELECT * FROM dbo.Trash")
    row = cursor.fetchone()
    while row:
        print (str(row[0]))
        row = cursor.fetchone()

In [ ]:
# Execute main function
if __name__ == '__main__':
    main()

# PostGre SQL Operations

In [ ]:
import os
pgserver = os.getenv("PGSERVER")
pgdatabase = os.getenv("PGDATABASE")
pgusername = os.getenv("PGUSERNAME")
pgpassword = os.getenv("PGPWD")

In [ ]:
import os
import psycopg2

# Update connection string information from env variables
pgserver = os.getenv("PGSERVER")
pgdatabase = os.getenv("PGDATABASE")
pgusername = os.getenv("PGUSERNAME")
pgpassword = os.getenv("PGPWD")
sslmode = "require"

# Construct connection string with psycopg2
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(pgserver, pgusername, pgdatabase, pgpassword, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")

cursor = conn.cursor()

In [ ]:
# Get first x 10 records of PostGre public.Trash table
cursor.execute("SELECT * FROM public.Trash")
row = cursor.fetchone()

for k in range(0,10):
    for i in range(0,5):
        print (str(row[i]))
    row = cursor.fetchone()

In [ ]:
# INSERT REQUEST: point parameter
def trashInsert(the_geom,cursor,connexion):
    point = 'POINT(' + str(the_geom.coords.xy[0][0]) + ' ' + str(the_geom.coords.xy[1][0]) + ')'
    cursor.execute("INSERT INTO public.trash (id, id_ref_campaign_fk,the_geom, elevation, id_ref_trash_type_fk,brand_type ) VALUES (DEFAULT, 'fb367e0e-9b56-4418-9b80-5acd5c239754',%s,'610.109375',1,%s) RETURNING id;", (point,'oasis'))
    connexion.commit()

In [ ]:
# Testing Trash Insert with shapely python Point in PostGre
from shapely.geometry import Point
geopoint = Point(0.0,1.0)
trashInsert(geopoint,cursor,conn)

# Trash End to End - PostGre

In [ ]:
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobClient
import json 
import os
import pyodbc
import subprocess

# Extract & Transform: IA

In [ ]:
''' blobContainer create a name_list of blobs within container'''
''' params are container name (no full url) & storage conn string'''
def blobInContainer(connection_s,container_n):
    campaign_container = ContainerClient.from_connection_string(conn_str=connection_s, container_name=container_n)
    blob_list = campaign_container.list_blobs()
    blob_names_list = []
    for blob in blob_list:
        blob_names_list.append(blob.name)
    return blob_names_list

''' blobInfos provides basic information about a blob object'''
''' params are blob_name only (no full url) & storage conn string'''
def blobInfos(connection_s,container_n,blob_n):
    blob_video = BlobClient.from_connection_string(conn_str=connection_s,container_name=container_n, blob_name=blob_n)
    blob_video_url = blob_video.url
    blob_video_prop = blob_video.get_blob_properties()
    blob_video_prop_keys = blob_video_prop.keys()
    print("blob name:",blob_n)
    print("blob URL:",blob_video_url)
    print("blob properties:", blob_video_prop)
    print("blob properties keys:", blob_video_prop_keys)


''' downloadBlob from Azure to local file system'''
''' parameter is a blob client object from azure storage sdk'''
def downloadBlob(blobclient):
    with open("/tmp/"+blobclient.blob_name, "wb") as my_blob_dl:
        blob_data = blobclient.download_blob()
        blob_data.readinto(my_blob_dl)
    print("Blob %s downloaded" %blobclient.blob_name)
    print("Blob path: /tmp/%s" %blobclient.blob_name)
    path = "/tmp/"+blobclient.blob_name
    return path


''' getPrediction function sends curl request to Surfrider AI'''
''' video name is the name of a locally downloaded video from Azure'''
''' video name is passed as an argument to curl_request script '''
''' curl_request script sends actual POST request to Surfrider AI'''
def getPrediction(video_name):
    curl_request_script = ['./curl_request_param.sh',video_name]
    output = []
    request_answer = subprocess.Popen(curl_request_script, stdout=subprocess.PIPE)
    i = 0
    for line in request_answer.stdout:
        print(line)
        output.append(line)
    return output


''' jsonPrediction cast a prediction from getPrediction function '''
''' from a list prediction to a string then dictionnary with json_loads '''
def jsonPrediction(pred):
    string_prediction = str(pred[0])[2:-3] #removing 2 x first and 3 last characters of pred
    json_prediction = json.loads(string_prediction)
    return json_prediction


''' getTrashLabel return label from a frame_to_box'''
def getTrashLabel(frame_2_box):
    return frame_2_box['label']

''' mapLabelTrashId is a switch that converts label to TrashId'''
''' param is label that comes from AI predictions dictionnary jsonPrediction'''
def mapLabel2TrashId(label):
    switcher = { 
    "Fishing or Hunting":"89B44BAA-69AA-4109-891A-128E012E7E07",
    "Food Packaging":"185FEFA2-EEF2-47A8-873E-26032A4BB3C3",
    "Unknown":"BB4DEA69-218A-40CC-A000-2AE17C37152C",
    "Industrial or Construction Debris":"2A863E38-E5D0-455F-87CE-2B75DA29F59A",
    "fragments":"ED401B92-DC24-44C0-A52A-34CE831092BF",
    "Agricultural Waste":"36B2AFEB-7A7C-44B5-A790-5E5C73BA144D",
    "others":"4BEC18FC-BC48-45B7-AFDA-6BA96BD80921",
    "Common Household Items":"C68E90CF-6E65-4474-BC60-72E1C8513F55",
    "plastic":"6961D0DB-928C-419E-9985-98EEEAF552C7",
    "bottles":"9780940B-D06C-4AAB-8003-AB914981E87A",
    "Drinking Bottles":"BCF549A8-AECD-4BC9-B9B8-B94A8F3758D5",
    "Unknown10":"BC7BB564-BE04-4B4B-9913-FF69780B93A6"
    } 
    return switcher.get(label, "nothing")

In [ ]:
# blob storage connection string
connection_string = os.getenv("CONN_STRING")

# get list of blobs in container campaign0
campaign_container_name = 'campaign0'
#campaign_container_name = 'campaign1'
blobs_campaign0 = blobInContainer(connection_string,campaign_container_name)
print(blobs_campaign0)

# get infos of blob video vid1-4K.MP4,vid1-4K2.MP4,vid1-4K3.mp4,vid1-4K4.mp4,19022020_Adour_6.mp4
#    blob_video_name = '19022020_Adour_6.mp4'
blob_video_name = 'goproshort-480p.mov'   
blobInfos(connection_string,campaign_container_name,blob_video_name)

# download blob video
blob_video0 = BlobClient.from_connection_string(conn_str=connection_string,container_name=campaign_container_name, blob_name=blob_video_name)
downloadBlob(blob_video0)

# get predictions from AI on vid1-4K4.MP4
prediction = getPrediction(blob_video_name)

# cast prediction to JSON/Dictionnary format
json_prediction = jsonPrediction(prediction)
json_prediction

In [ ]:
json_prediction['detected_trash'][0]

# Extract & Transform: GPS

In [ ]:
#https://www.cyberciti.biz/faq/python-command-line-arguments-argv-example/

In [ ]:
# before getting GPX, make sure GPS Data is included within GoPro File with ffprobe check on
# Check whether mp4 to move still contains GPS data ...

### Create GPX from GoPro video

In [ ]:
# Get GPX from Video
def goproToGPX(video_name):
    gopro2gpx_script = ['./gopro2gpx_param.sh',video_name]
    result = subprocess.Popen(gopro2gpx_script, stdout=subprocess.PIPE)
    output = []
    i = 0
    for line in result.stdout:
        print(line)
        output.append(line)
    path='/tmp/'+video_name+'.gpx'
    return path

In [ ]:
video_name = '28022020_Boudigau_4.mp4'
gpx_path = goproToGPX(video_name)
#gpx_path = goproToGPX('GH010010.MP4')

### Create GPS points from GPX

In [ ]:
# Parse GPX file
import gpxpy
import gpxpy.gpx

gpx_file = open(gpx_path,'r',encoding='utf-8')
gpx_data = gpxpy.parse(gpx_file) # data from parsed gpx file

In [ ]:
# Create GPX points list
''' gpsPointList function extract gps points from gpx file '''
''' gpxdata is a gpxpy object that returns data from a parsed gpx file'''
''' gpsPointList return a list of dictionnary points with Time, Lat, Long, Elev'''
def gpsPointList(gpxdata):
    point_list = []
    for track in gpxdata.tracks:
        for segment in track.segments: 
            for point in segment.points:
                point_info = {'Time':point.time,'Latitude':point.latitude,'Longitude':point.longitude,'Elevation':point.elevation}
                point_list.append(point_info)
    return point_list

In [ ]:
gpsPoints = gpsPointList(gpx_data)
print(len(gpsPoints))
for gps in gpsPoints:
    print(gps)

In [ ]:
timestampDelta = gpsPoints[len(gpsPoints)-1]['Time'] - gpsPoints[0]['Time']
timestampDelta

### Get Video Metadata info

In [ ]:
import json
import subprocess

#===============================
def getMediaInfo(mediafile):
    cmd = "mediainfo --Output=JSON %s"%(mediafile)
    proc = subprocess.Popen(cmd, shell=True,stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, stderr = proc.communicate()
    data = json.loads(stdout)
    return data

#===============================
def getDuration(mediafile):
    data = getMediaInfo(mediafile)
    duration = float(data['media']['track'][0]['Duration'])
    return duration

In [ ]:
video_duration = getDuration('/tmp/'+video_name)
print("Video duration in second from metadata:",video_duration)

### Fill in missing GPS timestamps

In [ ]:
import datetime
from datetime import datetime
from datetime import timedelta

def createTime(time):
    new_time = time
    new_time = new_time + timedelta(seconds=1)
    return new_time

def createLatitude(lat1,lat2):
    new_latitude = (lat1+lat2)/2
    new_latitude = round(new_latitude,6)
    return new_latitude

def createLongitude(long1,long2):
    new_longitude = (long1+long2)/2
    new_latitude = round(new_longitude,6)
    return new_longitude

def createElevation(elev1,elev2):
    new_elevation = (elev1+elev2)/2
    new_elevation = round(new_elevation,6)
    return new_elevation

In [ ]:
def fillGPS(inputGPSList,videoLength):
    filledGps = inputGPSList.copy()
    gps_length = len(filledGps)
    iteration_length = int((filledGps[gps_length-1]['Time'] - filledGps[0]['Time']).total_seconds())
    #print(iteration_length)
    ## this section output a filled gps list of length iteration_length+1 = Delta T between last gps timestamp and first one
    i = 0
    while i < (iteration_length):
#        print(i)
        delta = filledGps[i+1]['Time']-filledGps[i]['Time']
        delta = int(delta.total_seconds())
        if delta > 1: # adding a newly created element at index i+1
            #print(i)
            missing_time = createTime(filledGps[i]['Time'])
            missing_latitude = createLatitude(filledGps[i]['Latitude'],filledGps[i+1]['Latitude'])
            missing_longitude = createLongitude(filledGps[i]['Longitude'],filledGps[i+1]['Longitude'])
            missing_elevation = createElevation(filledGps[i]['Elevation'],filledGps[i+1]['Elevation'])
            new_gps = {'Time':missing_time,'Latitude':missing_latitude,'Longitude':missing_longitude,'Elevation':missing_elevation}
            filledGps.insert(i+1,new_gps)
        i = i+1
    ## this section add missing point at the end of the list, in case filledGps initial Delta time length is less than actual video length
    if len(filledGps) < videoLength:
        j = 0
        while len(filledGps) < videoLength:
            filledGps.insert(len(filledGps),filledGps[len(filledGps)-1])
            j = j+1

    return filledGps

In [ ]:
video_duration_sup = int(video_duration)+1
gpsPointsFilled = fillGPS(gpsPoints,video_duration_sup)

### Transform LongLat to Point()

In [ ]:
from shapely.geometry import Point

def longLat2shapePoint(gpsLongLatPoint):
    gpsShapePoint = {'Time':gpsLongLatPoint['Time'],'the_geom':Point(gpsLongLatPoint['Longitude'],gpsLongLatPoint['Latitude']),'Elevation':gpsLongLatPoint['Elevation']}
    return gpsShapePoint

def longLat2shapeList(gpsLongLagList):
    gpsShapeList = []
    for gpsPoint in gpsLongLagList:
        gpsShapePoint = longLat2shapePoint(gpsPoint)
        gpsShapeList.append(gpsShapePoint)
    return gpsShapeList

In [ ]:
# Convert gpsPoints to gpsShapePoints
gpsShapePointsFilled = longLat2shapeList(gpsPointsFilled)

In [ ]:
# Comparing point in gpsPointsFilled and in gpsShapePointsFilled
for i in range(0,20):
    print(gpsPoints[i])
    print(gpsShapePointsFilled[i]['the_geom'])

In [ ]:
from functools import partial
import pyproj
from shapely.ops import transform
from tqdm import tqdm

def geometryTransfo(gpsShapePoint):
    project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:2154')) # destination coordinate system

    geo1 = gpsShapePoint['the_geom']
    geo2 = transform(project,geo1)

    return geo2

def gps2154(gpsShapePointsFilled):
    gps2154Points = []
    for point in tqdm(gpsShapePointsFilled):
        geo2154 = geometryTransfo(point)
        gps2154Point = {'Time':point['Time'],'the_geom':geo2154,'Elevation':point['Elevation']}
        gps2154Points.append(gps2154Point)
    return gps2154Points

In [ ]:
# Testing geometryTransfo() on single point
geo2 = geometryTransfo(gpsShapePointsFilled[0])
print(gpsShapePointsFilled[0]['the_geom'])
print(geo2)

In [ ]:
# Testing gps2154() on all gpsShapePointsFilled
import warnings
warnings.filterwarnings('ignore')

gps2154PointsFilled = gps2154(gpsShapePointsFilled)

In [ ]:
# Checking Geo before and then 2154
for i in range (0,10):
    print(gpsShapePointsFilled[i]['the_geom'])
    print(gps2154PointsFilled[i]['the_geom'])

In [ ]:
for point in gps2154PointsFilled:
    print(point)
    print(point['the_geom'])

# Load: To PostGRE

In [111]:
import psycopg2

# Update connection string information from env variables
pgserver = os.getenv("PGSERVER")
pgdatabase = os.getenv("PGDATABASE")
pgusername = os.getenv("PGUSERNAME")
pgpassword = os.getenv("PGPWD")
sslmode = "require"

# Construct connection string with psycopg2
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(pgserver, pgusername, pgdatabase, pgpassword, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")

cursor = conn.cursor()

Connection established


In [ ]:
# INSERT REQUEST: point parameter
def trashInsert(the_geom,cursor,connexion):
    point = 'POINT(' + str(the_geom.coords.xy[0][0]) + ' ' + str(the_geom.coords.xy[1][0]) + ')'
    cursor.execute("INSERT INTO public.trash (id, id_ref_campaign_fk,the_geom, elevation, id_ref_trash_type_fk,brand_type ) VALUES (DEFAULT, '003d8675-29e3-4cde-a315-095ac2ec80bc',%s,'610.109375',1,%s) RETURNING id;", (point,'icetea'))
    connexion.commit()

In [ ]:
points = gps2154PointsFilled
points[0]['the_geom']

In [ ]:
trashInsert(points[0]['the_geom'],cursor,conn)

In [ ]:
# Label to TrashId PGSQL Data Model
def mapLabel2TrashIdPG(label):
    switcher = { 
        "others":"1", #"autre dechet" in PG Data Model mapped to IA "others" label
        "dechet agricole":"2",
        "bottles":"3", #"bouteille boisson" in PG Data Model mapped to IA "bottles" label
        "fragments":"4",#"industriel ou construction in PG Data Model mapped to IA "fragments" label
        "peche et chasse":"5",
        "emballage alimentaire":"6",
        "objet vie courante":"7",
        "autres dechets +10":"8"
    } 
    return switcher.get(label, "nothing")

In [112]:
# INSERT REQUEST: point parameter + trashTypeId
def trashInsert(gps2154Point,trashTypeId,cursor,connexion):
    point = 'POINT(' + str(gps2154Point['the_geom'].coords.xy[0][0]) + ' ' + str(gps2154Point['the_geom'].coords.xy[1][0]) + ')'
    elevation = gps2154Point['Elevation']
    cursor.execute("INSERT INTO public.trash (id, id_ref_campaign_fk,the_geom, elevation, id_ref_trash_type_fk,brand_type ) VALUES (DEFAULT, '003d8675-29e3-4cde-a315-095ac2ec80bc',%s,%s,%s,%s) RETURNING id;", (point,elevation,trashTypeId,'icetea'))
    connexion.commit()

### Trash INSERT using GPS point0

In [113]:
# Point0
gps2154Point0 = points[0]

# Trash0 from AI prediction0
pred0 = json_prediction['detected_trash'][0]
label0 = getTrashLabel(pred0)
trashType0 = mapLabel2TrashIdPG(label0)

In [114]:
# INSERT AI Trash0 with GPS Point 0 within PG
trashInsert(gps2154Point0,trashType0,cursor,conn)

### Trash INSERT using GPS dummy function

In [115]:
# Dummy Function to give GPS point to Detected Trash, based on TrashId
def trashGPS(trashId,gps2154Points):
    length = len(gps2154Points)+1
    gpsIndex = trashId % length
    return gpsIndex

In [116]:
# GPS coordinate for trashId0
trashTypeId0 = pred0['id']
gpsIndexId0 = trashGPS(trashTypeId0,gps2154PointsFilled)
gpsTrashTypeId0 = gps2154PointsFilled[gpsIndexId0]

# Trash0 from AI prediction0
pred0 = json_prediction['detected_trash'][0]
label0 = getTrashLabel(pred0)
trashType0 = mapLabel2TrashIdPG(label0)

In [117]:
# INSERT AI Trash0 with GPS from dummy trashGPS function
trashInsert(gpsTrashTypeId0,trashType0,cursor,conn)